In [45]:
from pyspark.sql.types import StructType,StringType,StructField,IntegerType

In [3]:
orders_schema= StructType([
    StructField("id",StringType()),
    StructField("created_at",StringType()),
    StructField("discount",StringType()),
    StructField("product_id",StringType()),
    StructField("quantity",StringType()),
    StructField("subtotal",StringType()),
    StructField("tax",StringType()),
    StructField("total",StringType()),
    StructField("user_id",StringType())
])

In [11]:
orders_df=spark.read \
.schema(orders_schema) \
.option("delimiter",",") \
.option("header",True) \
.csv("file:///home/sanjay/SparkTutorial/data/csv/orders.csv")

In [12]:
orders_df.createOrReplaceTempView("orders")
df_orders=spark.sql("select * from orders")
df_orders.drop()
df_orders.show(10,False)

+---+-----------------------------+-----------------+----------+--------+------------------+----+------------------+-------+
|id |created_at                   |discount         |product_id|quantity|subtotal          |tax |total             |user_id|
+---+-----------------------------+-----------------+----------+--------+------------------+----+------------------+-------+
|1  |2019-02-11T21:40:27.892+05:30|null             |14        |2       |37.648145389078365|2.07|39.718145389078366|1      |
|2  |2018-05-15T08:04:04.580+05:30|null             |123       |3       |110.93145648834248|6.1 |117.0376564084763 |1      |
|3  |2019-12-06T22:22:48.544+05:30|6.416679208849759|105       |2       |52.723521442619514|2.9 |55.62208681964182 |1      |
|4  |2019-08-22T16:30:42.392+05:30|null             |94        |6       |109.21864156655383|6.01|115.2207354961295 |1      |
|5  |2018-10-10T03:34:47.309+05:30|null             |132       |5       |127.88197029833711|7.03|134.94192935296473|1      |


# Without Hard Coding(defining directories in config files)

In [13]:
import configparser

In [15]:
config=configparser.ConfigParser()
config.read("./configs/config.ini")
inputLocation= config.get('paths','inputLocation')

In [16]:
orders_df2=spark.read \
.schema(orders_schema) \
.option('delimiter',',') \
.option('header',True) \
.csv(inputLocation)
orders_df2.show(10,False)

+---+-----------------------------+-----------------+----------+--------+------------------+----+------------------+-------+
|id |created_at                   |discount         |product_id|quantity|subtotal          |tax |total             |user_id|
+---+-----------------------------+-----------------+----------+--------+------------------+----+------------------+-------+
|1  |2019-02-11T21:40:27.892+05:30|null             |14        |2       |37.648145389078365|2.07|39.718145389078366|1      |
|2  |2018-05-15T08:04:04.580+05:30|null             |123       |3       |110.93145648834248|6.1 |117.0376564084763 |1      |
|3  |2019-12-06T22:22:48.544+05:30|6.416679208849759|105       |2       |52.723521442619514|2.9 |55.62208681964182 |1      |
|4  |2019-08-22T16:30:42.392+05:30|null             |94        |6       |109.21864156655383|6.01|115.2207354961295 |1      |
|5  |2018-10-10T03:34:47.309+05:30|null             |132       |5       |127.88197029833711|7.03|134.94192935296473|1      |


# Without Hard Coding(defining schema in config and function)

In [54]:
config.read('./configs/config.ini')
schemaFromConf=config.get('schemas','ingestSchema')

In [70]:
def read_schema(schema):
	d_types={"String":StringType(),"Int":IntegerType()}
	all_values=list(schema.split(","))
	sch=StructType()
	for each in all_values:
		value=list(each.split(" "))
		sch.add(value[0],d_types.get(value[1]),True)
	return sch
		

In [71]:
orders_schema2=read_schema(schemaFromConf)

In [73]:
orders_df3=spark.read \
.schema(orders_schema2) \
.option('delimiter',',') \
.option('header',True) \
.csv(inputLocation)
orders_df3.show(10,False)

+---+-----------------------------+-----------------+----------+--------+------------------+----+------------------+-------+
|id |created_at                   |discount         |product_id|quantity|subtotal          |tax |total             |user_id|
+---+-----------------------------+-----------------+----------+--------+------------------+----+------------------+-------+
|1  |2019-02-11T21:40:27.892+05:30|null             |14        |2       |37.648145389078365|2.07|39.718145389078366|1      |
|2  |2018-05-15T08:04:04.580+05:30|null             |123       |3       |110.93145648834248|6.1 |117.0376564084763 |1      |
|3  |2019-12-06T22:22:48.544+05:30|6.416679208849759|105       |2       |52.723521442619514|2.9 |55.62208681964182 |1      |
|4  |2019-08-22T16:30:42.392+05:30|null             |94        |6       |109.21864156655383|6.01|115.2207354961295 |1      |
|5  |2018-10-10T03:34:47.309+05:30|null             |132       |5       |127.88197029833711|7.03|134.94192935296473|1      |
